In [1]:
import json
import boto3
import pandas as pd
import ast

from utils.analysis_utils import process_analysis_in_parallel, process_analysis, format_ligne
from utils.database import get_field_value
from pymongo import MongoClient
import certifi
import importlib

In [2]:
# TODO: Remove

import importlib
import utils.analysis_utils
# import utils.database

# Reload the modules
importlib.reload(utils.analysis_utils)
# importlib.reload(utils.database)

# Now you can import your functions
from utils.analysis_utils import process_analysis_in_parallel, process_analysis, format_ligne
# from utils.database import get_field_value


In [3]:
_secrets_manager_client = boto3.client("secretsmanager", region_name="eu-west-3")

_secrets = json.loads(
    _secrets_manager_client.get_secret_value(
        SecretId=f"Prod/alloreview"
    )["SecretString"]
)
MONGO_CONNECTION_STRING = (
    "mongodb+srv://alloreview:{}@feedbacksdev.cuwx1.mongodb.net".format(
        _secrets["mongodb"]["password"]
    )
)
mongo_client = MongoClient(MONGO_CONNECTION_STRING,tlsCAFile=certifi.where())

collection = mongo_client['feedbacks_db']['feedbacks_Prod']

In [4]:
BRAND = 'ditp_analysis'

BRAND_DESCR = '''
Feedbacks are from French public services.
'''

In [27]:
from_mongo = pd.DataFrame(list(collection.aggregate([
    {
        '$match': {
            'brand': BRAND,
            'extractions': {
                '$exists': True, 
                '$not': {'$size': 0}  # assure que 'extractions' n'est pas vide
            },
            'splitted_analysis_v2': {
                '$exists': True, 
                '$not': {'$size': 0}  # assure que 'splitted_analysis_v2' n'est pas vide
            },
            'extractions': {
                '$not': {
                    '$elemMatch': {
                        'elementary_subjects': {'$exists': True}
                    }
                }  # exclut les documents contenant 'elementary_subjects' dans au moins un élément de 'extractions'
            }
        }
    },
    { "$sample" : { "size": 50 } }  # échantillonne 500 documents
])))

from_mongo.shape

(50, 88)

In [25]:
df = from_mongo[['_id', 'ecrit_le', 'splitted_analysis_v2', 'extractions', 'intitule_structure_1', 'intitule_structure_2', 'tags_metiers', 'pays', 'verbatims']]


df['brand_context'] = df.apply(format_ligne, axis=1)
subdf = df[['_id', 'extractions', 'brand_context']]

print(f"Number of rows pending elementary_subjects processing: {subdf.shape[0]}.")

Number of rows pending elementary_subjects processing: 500.


/var/folders/6h/k0ccs7cd2yjfr44gzm0lx11w0000gn/T/ipykernel_90009/423548503.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['brand_context'] = df.apply(format_ligne, axis=1)


In [12]:
tableau_extraction = subdf.explode('extractions')[['_id', 'extractions', 'brand_context']]
tableau_extraction

,_id,extractions,brand_context
0,ditp_analysis/3585826,"{'sentiment': 'NEGATIVE', 'extraction': 'Absen...",Feedbacks are from French public services.\nIn...
0,ditp_analysis/3585826,"{'sentiment': 'NEGATIVE', 'extraction': 'Fonct...",Feedbacks are from French public services.\nIn...
0,ditp_analysis/3585826,"{'sentiment': 'NEGATIVE', 'extraction': 'Manqu...",Feedbacks are from French public services.\nIn...
6,ditp_analysis/4124483,"{'sentiment': 'NEGATIVE', 'extraction': 'Diffi...",Feedbacks are from French public services.\nIn...
6,ditp_analysis/4124483,"{'sentiment': 'NEGATIVE', 'extraction': 'Probl...",Feedbacks are from French public services.\nIn...
...,...,...,...
469,ditp_analysis/276187,"{'sentiment': 'SUGGESTION', 'extraction': 'Sur...",Feedbacks are from French public services.\nIn...
472,ditp_analysis/1930951,"{'sentiment': 'NEGATIVE', 'extraction': 'Probl...",Feedbacks are from French public services.\nIn...
472,ditp_analysis/1930951,"{'sentiment': 'NEGATIVE', 'extraction': 'Probl...",Feedbacks are from French public services.\nIn...
472,ditp_analysis/1930951,"{'sentiment': 'NEGATIVE', 'extraction': 'Erreu...",Feedbacks are from French public services.\nIn...


In [13]:
# TODO: Remove

import importlib
import utils.analysis_utils
# import utils.database

# Reload the modules
importlib.reload(utils.analysis_utils)
importlib.reload(utils.database)

# Now you can import your functions
from utils.analysis_utils import process_analysis_in_parallel, process_analysis, format_ligne, classify_extraction_with_topics, get_elementary_subjects_for_part_of_feedback, mapping_duplicates, rename_duplicates
from utils.database import update_feedbacks_in_mongo

In [16]:
def process_row(row):
    row['extractions'], row['elementary_subjects'] = get_elementary_subjects_for_part_of_feedback(
        extractions=row['extractions'],
        language='french',
        brand_name='ditp_analysis',
        brand_context=row['brand_context'],
        model='gpt-4o-mini',
        should_update_mongo=False
    )
    return row

### For one feedback

In [15]:
# Sample one distinct '_id'
sampled_id = tableau_extraction['_id'].sample(1).iloc[0]

# Filter the dataframe based on the sampled '_id'
sample_tableau_extraction = tableau_extraction[tableau_extraction['_id'] == sampled_id]
sample_tableau_extraction

,_id,extractions,brand_context
14,ditp_analysis/4702850,"{'sentiment': 'NEGATIVE', 'extraction': 'Saisi...",Feedbacks are from French public services.\nIn...
14,ditp_analysis/4702850,"{'sentiment': 'NEGATIVE', 'extraction': 'PV re...",Feedbacks are from French public services.\nIn...
14,ditp_analysis/4702850,"{'sentiment': 'NEGATIVE', 'extraction': 'Conte...",Feedbacks are from French public services.\nIn...
14,ditp_analysis/4702850,"{'sentiment': 'NEGATIVE', 'extraction': 'Courr...",Feedbacks are from French public services.\nIn...
14,ditp_analysis/4702850,"{'sentiment': 'NEGATIVE', 'extraction': 'Nombr...",Feedbacks are from French public services.\nIn...
14,ditp_analysis/4702850,"{'sentiment': 'NEGATIVE', 'extraction': 'Servi...",Feedbacks are from French public services.\nIn...
14,ditp_analysis/4702850,"{'sentiment': 'NEGATIVE', 'extraction': 'Rende...",Feedbacks are from French public services.\nIn...
14,ditp_analysis/4702850,"{'sentiment': 'POSITIVE', 'extraction': 'Coord...",Feedbacks are from French public services.\nIn...
14,ditp_analysis/4702850,"{'sentiment': 'NEGATIVE', 'extraction': 'Prélè...",Feedbacks are from French public services.\nIn...
14,ditp_analysis/4702850,"{'sentiment': 'NEGATIVE', 'extraction': 'Reten...",Feedbacks are from French public services.\nIn...


In [ ]:

# Apply the function to each row
sample_tableau_extraction = sample_tableau_extraction.apply(process_row, axis=1)
tableau_extraction = sample_tableau_extraction

### For the table

In [17]:
from concurrent.futures import ThreadPoolExecutor
# Fonction pour appliquer la parallélisation sur tout le dataframe
def parallelize_dataframe(df, func, workers=10):
    with ThreadPoolExecutor(max_workers=workers) as executor:
        results = list(executor.map(func, [row for _, row in df.iterrows()]))
    return pd.DataFrame(results)

# Utilisation de la parallélisation pour traiter le dataframe
tableau_extraction = parallelize_dataframe(tableau_extraction, process_row)
tableau_extraction.head(10)

,_id,extractions,brand_context,elementary_subjects
0,ditp_analysis/3585826,"{'sentiment': 'NEGATIVE', 'extraction': 'Absen...",Feedbacks are from French public services.\nIn...,[]
0,ditp_analysis/3585826,"{'sentiment': 'NEGATIVE', 'extraction': 'Fonct...",Feedbacks are from French public services.\nIn...,[]
0,ditp_analysis/3585826,"{'sentiment': 'NEGATIVE', 'extraction': 'Manqu...",Feedbacks are from French public services.\nIn...,[]
6,ditp_analysis/4124483,"{'sentiment': 'NEGATIVE', 'extraction': 'Diffi...",Feedbacks are from French public services.\nIn...,[Assistance : Difficulté à Contacter le Servic...
6,ditp_analysis/4124483,"{'sentiment': 'NEGATIVE', 'extraction': 'Probl...",Feedbacks are from French public services.\nIn...,[Amendes : Difficulté de Contact pour Paiement]
6,ditp_analysis/4124483,"{'sentiment': 'NEGATIVE', 'extraction': 'Probl...",Feedbacks are from French public services.\nIn...,[Service : Difficulté de Contact pour Contesta...
6,ditp_analysis/4124483,"{'sentiment': 'NEGATIVE', 'extraction': 'Appel...",Feedbacks are from French public services.\nIn...,[Assistance : Difficulté à Joindre le Service ...
6,ditp_analysis/4124483,"{'sentiment': 'NEGATIVE', 'extraction': 'Appel...",Feedbacks are from French public services.\nIn...,[Assistance : Difficulté à Joindre le Service ...
6,ditp_analysis/4124483,"{'sentiment': 'NEGATIVE', 'extraction': 'Appel...",Feedbacks are from French public services.\nIn...,[Assistance : Difficulté à Contacter le Servic...
6,ditp_analysis/4124483,"{'sentiment': 'NEGATIVE', 'extraction': 'Véhic...",Feedbacks are from French public services.\nIn...,[Assistance : Difficulté à Contacter le Servic...


### Regroupe

In [19]:
regrouped_df = tableau_extraction.groupby('_id').agg({'extractions': list}).reset_index()
regrouped_df.head(5)

,_id,extractions
0,ditp_analysis/1322293,"[{'sentiment': 'NEGATIVE', 'extraction': 'Serv..."
1,ditp_analysis/1930951,"[{'sentiment': 'NEGATIVE', 'extraction': 'Prob..."
2,ditp_analysis/276187,"[{'sentiment': 'NEGATIVE', 'extraction': 'Prop..."
3,ditp_analysis/279177,"[{'sentiment': 'POSITIVE', 'extraction': 'Expé..."
4,ditp_analysis/2982465,"[{'sentiment': 'NEGATIVE', 'extraction': 'Prob..."
5,ditp_analysis/3018974,"[{'sentiment': 'NEGATIVE', 'extraction': 'Serv..."
6,ditp_analysis/3059026,"[{'sentiment': 'NEGATIVE', 'extraction': 'Prob..."
7,ditp_analysis/3071452,"[{'sentiment': 'NEGATIVE', 'extraction': 'Dema..."
8,ditp_analysis/3294882,"[{'sentiment': 'NEGATIVE', 'extraction': 'Reve..."
9,ditp_analysis/3300259,"[{'sentiment': 'NEGATIVE', 'extraction': 'Retr..."


## Rename duplicates

In [20]:
# Fonction pour appliquer rename_duplicates sur une ligne
def process_row(row):
    return rename_duplicates(row)

### For only one row

In [60]:
# Suppose you want to apply process_row on line 0
row = regrouped_df['extractions'].iloc[0]

# Process that single row using process_row
processed_row = process_row(row)

[{'role': 'user', 'content': '\nYou are given a list of topics. Your task is to identify and remove duplicates or highly similar topics, then return a structured JSON object following these rules:\n\n1. **Final Topics as Keys**: Each unique final topic should be a key in the JSON object.\n2. **Merged Topics as Values**: For each key, include a list of all original topics that were merged into that final topic. The list must include the final topic itself.\n\n### Guidelines:\n- **Clarity and Detail**: Keep the most detailed and precise version of any duplicate or similar topics as the final topic.\n- **Similarity Threshold**: Merge topics that express the same concept using different phrasing or are at least 70% similar.\n- **Exact Duplicates**: Eliminate exact duplicates and merge them under the most descriptive version.\n- **Output Format**: Return the result as a JSON object, with the final topics as keys and the merged topics as their corresponding values.\n\n### Example Input:\n["A

### For every rows

In [21]:
import concurrent.futures

# Utiliser ThreadPoolExecutor dans un notebook pour éviter les erreurs de sérialisation
with concurrent.futures.ThreadPoolExecutor() as executor:
    regrouped_df['extractions'] = list(executor.map(process_row, regrouped_df['extractions']))

{'Unknown Topic': ['Unknown Topic']}
{"Changement d'Adresse : Refus sans Explication": ["Changement d'Adresse : Refus sans Explication"]}
{'Site Internet : Problèmes Techniques URSSAF': ['Site Internet : Problèmes Techniques URSSAF', 'Site Internet : Problèmes Techniques (Erreur 403)', 'URSSAF : Problèmes Techniques sur le Site']}
{"Service : Problèmes Techniques d'Accès": ["Service : Problèmes Techniques d'Accès", 'Service : Dysfonctionnement Technique', "CNAM : Problèmes d'Accès au Service en Ligne", "CNAM : Problème Technique d'Accès au Service"]}
{'Dépôt de Plainte : Accueil Positif': ['Dépôt de Plainte : Accueil Positif'], 'Accueil : Écoute et Bienveillance du Personnel': ['Accueil : Écoute et Bienveillance du Personnel', 'Service : Personnel : Écoute et Compréhension'], 'Unknown Topic': ['Unknown Topic']}
{'Paiement : Problème de Référence': ['Paiement : Problème de Référence pour Ordures Ménagères', 'Paiement : Problème de Référence', 'PayFip : Suggestion de Correction de Référe

### Update mongo - elementary_subjects list

In [63]:
regrouped_df['extractions']

0    [{'sentiment': 'NEGATIVE', 'extraction': 'Sais...
Name: extractions, dtype: object

### Update mongo - Feedbacks elementary_subjects

In [22]:
def bulk_update_from_dataframe(df, update_function):
    feedbacks_to_update = [
        {
            "id": row['_id'],  # Assuming '_id' column holds the document ID
            "updates": {
                "extractions": row['extractions'],  # Assuming 'extractions' is the field to be updated
            }
        }
        for _, row in df.iterrows()
    ]
    
    # Call the update function with the list of feedback updates
    update_function(feedbacks_to_update)

# Example usage with your dataframe `df`
bulk_update_from_dataframe(regrouped_df, update_feedbacks_in_mongo)


Updating 53 feedbacks
Matched: 53, Modified: 53


In [ ]:
# TODO:
# - add update_splitted_analysis
# - splitted_analysis = update_splitted_analysis(feedback_id, extractions, splitted_analysis_column)

In [34]:



res = process_analysis(
    feedback_id=row['_id'],
    extractions=row["extractions"],
    brand_context=row["brand_context"],
    model="gpt-4o-mini",
    brand_name=BRAND,
    language="french",
    extractions_column="extractions",
    should_update_mongo=True
)

res


TypeError: get_elementary_subjects_for_part_of_feedback() missing 4 required positional arguments: 'language', 'brand_name', 'brand_context', and 'model'

### For multiple feedbacks

In [7]:

importlib.reload(utils.analysis_utils)
from utils.analysis_utils import process_analysis_in_parallel

In [8]:
feedbacks_sample = subdf.sample(40)

In [9]:

process_analysis_in_parallel(
    feedbacks=feedbacks_sample,
    brand_name=BRAND,
    language='french',
    model="gpt-4o-mini",
    save_to_mongo=True
)

# res
# res = process_analysis_in_parallel(
#     feedbacks=feedbacks_sample,
#     brand_name=BRAND,
#     language='french',
#     model="gpt-4o-mini",
#     save_to_mongo=True
# )

# # res

Processing chunks:   0%|          | 0/2 [00:00<?, ?it/s]

Classification_result: {'topics': ['Suggestions : Amélioration de la Visibilité et du Suivi des Allocations Chômage'], 'justification': "Le feedback suggère une amélioration concernant la visibilité des noms des employeurs pour les employés CESU, ce qui pourrait faciliter leur travail d'actualisation. La catégorie 'Suggestions : Amélioration de la Visibilité et du Suivi des Allocations Chômage' est pertinente ici.", 'is_new_topic': False}
Classification_result: {'topics': ["Suivi de Dossier : Traitement Positif des Indemnités d'Arrêt Maladie"], 'justification': "Le feedback exprime une satisfaction concernant le traitement du dossier d'arrêt maladie et souligne le professionnalisme et l'aide reçue. Cela correspond à une expérience positive dans le cadre du suivi des indemnités d'arrêt maladie.", 'is_new_topic': False}
Classification_result: {'topics': ['Suggestions : Simplification des Procédures au Consulat', "Suggestions : Amélioration de l'Accessibilité pour le Premier Contact au Co

Processing chunks:  50%|█████     | 1/2 [09:23<09:23, 563.05s/it]

[check_duplicates()] invalid syntax (<string>, line 8)
Classification_result: {'topics': ['Opération Tranquillité Vacances : Satisfaction du Processus en Ligne'], 'justification': "Le feedback exprime une satisfaction concernant la facilité d'inscription en ligne pour le service 'Opération tranquillité vacances', ce qui correspond à une expérience positive liée à un processus en ligne. Cependant, aucune des catégories existantes ne couvre spécifiquement ce service. Il est donc nécessaire de créer une nouvelle catégorie.", 'is_new_topic': True}
Found 20 duplicates for negative
Found 30 duplicates for negative
Classification_result: {'topics': ["Service-public.fr : Appréciation Positive de l'Opération Tranquillité Vacances"], 'justification': "Le feedback exprime une appréciation positive concernant le service 'Opération tranquillité vacances', soulignant sa facilité d'accès et son utilité. Cependant, aucune des catégories existantes ne correspond spécifiquement à ce service. Il est donc

Processing chunks: 100%|██████████| 2/2 [12:03<00:00, 361.89s/it]

[check_duplicates()] invalid syntax (<string>, line 8)


[{'id': 'ditp_analysis/4457107',
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': 'Recensement intrusif et discriminatoire de genre',
    'text': 'Recencement trop intrusif et discriminatoire de genre',
    'elementary_subjects': ['INSEE : Perception de la Démarche de Recensement comme Intrusive']},
   {'sentiment': 'NEGATIVE',
    'extraction': 'Questions inutiles dans le recensement',
    'text': 'Beaucoup de questions inutiles pour un recensement',
    'elementary_subjects': ['INSEE : Perception de la Démarche de Recensement comme Intrusive']},
   {'sentiment': 'NEGATIVE',
    'extraction': "Impression d'interrogatoire de police",
    'text': "nous avons juste l'impression d'assister à un interrogatoire de police",
    'elementary_subjects': ['INSEE : Perception de la Démarche de Recensement comme Intrusive']},
   {'sentiment': 'SUGGESTION',
    'extraction': 'Prendre en compte plus de choix pour le genre',
    'text': "je m'étonne de n'avoir que 2 choix possible quand 

## Check elementary subjects that we generated

In [10]:
from utils.database import get_elementary_subjects

In [11]:
# Récupération des sujets élémentaires négatifs et positifs
negative_elementary_subjects = get_elementary_subjects(BRAND, 'negative')
positive_elementary_subjects = get_elementary_subjects(BRAND, 'positive')

# Affichage du nombre total de sujets élémentaires
total_subjects = len(negative_elementary_subjects) + len(positive_elementary_subjects)
print(f"Total elementary subjects: {total_subjects}")

# Sélection des 5 premiers sujets élémentaires négatifs et positifs
negative_subjects_preview = [x['elementary_subject'] for x in negative_elementary_subjects[:5]]
positive_subjects_preview = [x['elementary_subject'] for x in positive_elementary_subjects[:5]]

# Affichage des 5 premiers sujets élémentaires négatifs et positifs ensemble
print("First 5 negative elementary subjects:", negative_subjects_preview)
print("First 5 positive elementary subjects:", positive_subjects_preview)


Total elementary subjects: 559
First 5 negative elementary subjects: ['Assistance : Manque de Réponses et de Support', 'Récupération de Documents : Attente et Exigences Administratives', 'Système de Notation : Inexactitude des Retours Utilisateurs', 'Fonctionnement du Site : Service en Panne', 'Fonctionnement du Site : Impossibilité de Soumettre une Demande en Ligne']
First 5 positive elementary subjects: ['Service : Personnel : Accueil Positif', 'Service : Utilisation du Site : Simplicité et Convivialité', 'Service : Formulaire de Contact Disponible', 'Accès au Portefeuille Électronique : Difficulté à Trouver des Documents', "Problème de Reconnaissance de Véhicule dans le Système d'Immatriculation"]


In [12]:
[x['elementary_subject'] for x in negative_elementary_subjects[:]]

['Assistance : Manque de Réponses et de Support',
 'Récupération de Documents : Attente et Exigences Administratives',
 'Système de Notation : Inexactitude des Retours Utilisateurs',
 'Fonctionnement du Site : Service en Panne',
 'Fonctionnement du Site : Impossibilité de Soumettre une Demande en Ligne',
 'Distribution du Courrier : Retards et Dysfonctionnements',
 'Numérisation : Complications Administratives liées aux Algorithmes',
 'Inscription Consulaire : Complexité et Manque de Clarté',
 'Parcours Administratif : Complexité et Manque de Clarté',
 'Complexité Administrative : Déclaration de Travaux',
 'Inscription Consulaire : Problème de Numéro Numic',
 'Complexité Administrative : Soumission de Documents et Communication entre Services',
 'Procédures Administratives : Manque de Clarté et Complexité',
 'Demande de Passeport : Disponibilité des Créneaux de Rendez-vous',
 "Consulat : Horaires d'Ouverture Inadaptés",
 "Fonctionnement du Site : Manque de Clarté sur l'Interface Utilis

In [11]:
# row = subdf.sample().iloc[0]
# extractions = get_field_value(
#     feedback_id=row['_id'],
#     field_name='extractions',
# )
# res = classify_one_feedback(
#     feedback_id=row['_id'],
#     extractions=extractions,
#     model="gpt-4o-mini",
#     brand=BRAND,
#     brand_descr=BRAND_DESCR,
#     language="frenc
# h",
#     extractions_column="extractions",
#     update_mongo=False
# )

# # res